# Workshop: Access MICrONS connectomics data

The function of the nervous system arises out of a combination of the properties of individual neurons and the properties of how they are connected into a larger network. The central goal of <b>connectomics</b> is to produce complete maps of the connectivity of the nervous system with synaptic resolution and analyze them to better understand the organization, development, and function of the nervous system. 

<b>Electron Microscopy (EM) data enables morphological reconstruction of neurons and resolution of their synaptic connectivity </b>. The MICrONS dataset is one of the largest volume EM datasets currently available, and spans all layers of mouse visual cortex. We will be using this dataset to query the connectivity between neurons in the visual cortex. 

Further details about the dataset are available at [MICrONS Explorer](https://www.microns-explorer.org/), and detailed data access examples at [MICrONS Tutorials](https://tutorial.microns-explorer.org/).

For neuroglancer link, see: https://spelunker.cave-explorer.org/#!gs://microns-static-links/mm3/layer5_thick_tufted.json

<h3> Workshop aims</h3>
<ul>
    <li> Understand what are the major classes of cell types in cortex</li>
    <li> Understand the basics of how synaptic connectivity is measured in EM connectomics</li>
    <li> Look at the relationship between the structure of networks and cell type connectivity</li>
</ul>
    

<h4> This workshop will cover: </h4>
<ol> 
    <li>Reconstructions of individual neurons</li>
    <li>Connectivity of individual neurons</li>
    <li>Connectivity between cell types</li>
</ol>

## Connectome Annotation Versioning Engine (CAVE)

This tutorial walks through the key functions needed to access the MICrONS dataset programmatically and highlights key resources within it. While this tutorial is written for the MICrONS dataset specifically, the underlying technology (CAVE) is being used for multiple connectomics dataset. So, the interface presented here can be used to query them as well. 

The **CAVEclient** is a python library that facilitates communication with a CAVE system. For convenience, we also use the package **skeleton_plot** which handles rendering the precomputed skeletons.

In [ ]:
# @title For Colab: Run this cell once to install and load packages
%%capture
!pip install -q caveclient
!pip install -q skeleton_plot

In [ ]:
# Import packages
from caveclient import CAVEclient
import skeleton_plot as skelplot

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

### CAVE account setup

In order to manage server traffic, every user needs to create a CAVE account and download a user token to access CAVE's services programmatically. The CAVE infrastructure can be read about in more detail in the [MICrONS Nature Package](https://doi.org/10.1038/s41592-024-02426-z).

**A Google account (or Google-enabled account) is required to create a CAVE account.**

Go to: https://global.daf-apis.com/auth/api/v1/user/token to view a list of your existing tokens

If you have never made a token before, accept the terms of service and generate a token. Then copy the hash string and paste below

For Local environments
```
# Token Generation and Setup (Local)
CAVEclient.setup_token("https://global.daf-apis.com")

client = CAVEclient(datastack_name='minnie65_public')

```

In [ ]:
# @title Initialize CAVE
# @markdown Requires your unique user token. Input user token

client = CAVEclient()
user_token = '' # @param {type: "string"}

try:
  client.auth.save_token(token=user_token, overwrite=True)
  # Initialize a client for the production datastack.
  client = CAVEclient(datastack_name='minnie65_public')

  cg = client.chunkedgraph
  print('Client initiation successful!')
except:
  print('Oops! Token does not work')
  print('Go to: https://global.daf-apis.com/auth/api/v1/user/token to view a list of your existing tokens')
  print('If you have never made a token before: \n  1) go here: https://minnie.microns-daf.com/materialize/views/datastack/minnie65_public to accept terms of service \n  2) then go here https://global.daf-apis.com/auth/api/v1/create_token to create a new token.')

### Initialize CAVEclient with a datastack

Datasets in CAVE are organized as datastacks. These are a combination of an EM dataset, a segmentation and a set of annotations. The datastack for MICrONS public release is `minnie65_public`. When you instantiate your client with this datastack, it loads all relevant information to access it.

In [ ]:
datastack_name = "minnie65_public"

client = CAVEclient(datastack_name)

For each datastack, CAVE stores information about key data sources and parameters. These can be accessed through:

In [ ]:
client.info.get_datastack_info()

For instance, the synapse table is defined as `synapses_pni_2` and the cell body table as `nucleus_detection_v0`. 

## Part 1: Neuron morphology: representing neurons as skeletons

Often in thinking about neurons, you want to measure things along a linear dimension of a neuron.

However, the segmentation and meshes are a complex 3D shape that makes this non-trivial. There are methods for reducing the shape of a segmented neuron down to a linear tree like structure usually referred to as a **skeleton**. We have precalculated skeletons for a large number of cells in the dataset, and make the skeleton generation available on our server, on demand.

*Note*: The meshes you see in Neuroglancer are available to download through the python client `cloud-volume`, and can be loaded for analysis and visualization in other tools. This is useful for some types of analysis (and for making beautiful renders of neurons), but we will not cover that in this course.

Now we will select one neuron based on its `root_id`, the 18-digit number you see in neuroglancer, for example from:  https://spelunker.cave-explorer.org/#!gs://microns-static-links/mm3/layer5_thick_tufted.json

In [ ]:
# select the example cell from neuroglancer
root_id = 864691135572530981

# Query the skeleton from CAVEclient
sk_d = client.skeleton.get_skeleton(root_id, output_format="dict")

sk_d.keys() # print the keys of the skeleton dictionary object

Now we are going to define a convenience function to plot the skeleton

In [ ]:
# Color map for the different comparments. You may adjust these to your asthetic.
skel_color_map = {3: "firebrick", 4: "salmon", 2: "steelblue", 1: "olive"}


def plot_skeleton(sk_d: dict, plot_soma=True, ax=None):
    """Plots a skeleton.

    Args:
        sk: skeleton dict as returned from CAVEclient

    Returns:
        ax: plot axes
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 10), dpi=150)

    skelplot.plot_tools.plot_verts(
        np.array(sk_d["vertices"]),
        np.array(sk_d["edges"]),
        radius=np.array(sk_d["radius"]) / 1000 * 1,
        ax=ax,
        skel_colors=sk_d["compartment"],
        title="",
        skel_alpha=1.0,
        line_width=5,
        x="x",  # You can alter the projection direction by changing this to `y` or `z`
        y="y",  # You can alter the projection direction by changing this to `x` or `z`
        plot_soma=plot_soma,
        soma_node=sk_d["root"],
        color="darkslategray",
        soma_size=120,
        invert_y=True,
        skel_color_map=skel_color_map,
        x_min_max=None,
        y_min_max=None,
        capstyle="round",
        joinstyle="round",
    )

    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.axis("off")

    return ax

In [ ]:
# Plot skeleton in 2D
ax = plot_skeleton(sk_d)
plt.show()

### Morphological compartment names 

The compartment types defined at each vertex adhere to standardized conventions for neuronal morphologies saved as swc files (for more information, see here: www.neuromorpho.org). 
    
These conventions are as follows:
<ul> 
    <li> 0 - undefined </li>
    <li> 1 - soma (default color 'olive') </li>
    <li> 2 - axon (default color 'steelblue') </li>
    <li> 3 - (basal) dendrite (default color 'firebrick' red) </li>
    <li> 4 - apical dendrite </li>
    <li> 5+ - custom </li>
</ul>

In most of these neurons, distinctions were not made between basal or apical dendrites - therefore dendrites should almost exclusively map to compartment type "3".

<b>Discussion:</b> What information do you lose going from 3D neuron meshes (neuroglancer) to 2D skeleton projections? What becomes more clear?

## Part 2: annotation tables and generally useful information

A datastack has a large number of tables that can be intimidating to traverse at first. CAVE provides several ways to find the tables you may want use.  All database functions are under the `client.materialize` property. To see what tables are available, use the `get_tables function`:

In [ ]:
client.materialize.get_tables()

### Query 1: Querying cells and their types

The basic querying logic of CAVE is `client.materialize.query_table`. This accepts at least a table as parameter. Let's query the table of all automatically segmented nuclei:

In [ ]:
nucleus_table_name = client.info.get_datastack_info()["soma_table"]
nucleus_df = client.materialize.query_table(nucleus_table_name)
nucleus_df.head(5)

Every annotation table has at least one position column (here: `pt_position`) which serves as anchor to the segmentation. These positions are automatically associated to the segmentation using `pt_root_id`s which can be thought of segment or cell IDs. Beyond positions and their associated IDs, every table stores metadata. For instance, the nucleus table contains the `volume` of each cell body.

Every table has a description and metadata attached to it that describes how the data was generated, limitations of it, and papers to cite when using it:

In [ ]:
client.materialize.get_table_metadata(nucleus_table_name)

#### Location vs depth

As a first analysis, we will plot the depth location vs the size of each cell nucleus. `query_table` has additional parameters to modify the results and standardize returns that make such an analysis easier. 
* Using `desired_resolution` the resolution of all position columns can be defined in nanometers.
* Using `split_positions`, position columns are separate
* Using `select_columns` returns only the columns you know you are interested in. Try uncommenting this line to make the table more readable

In [ ]:
nucleus_df = client.materialize.query_table(nucleus_table_name,
                                            desired_resolution=[1000, 1000, 1000], # returned in um
                                            split_positions=True,
                                            # select_columns=['id','pt_root_id','volume','pt_position']
                                           )
nucleus_df.head(5)

The data is organized such that the `y` axis is roughly aligned with depth (there is a 5 degree tilt that can be adjusted with the [standard_transform](https://github.com/CAVEconnectome/standard_transform) package; we will ignore that here for simplicity). 

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=nucleus_df, x="volume", y="pt_position_y", size=1, edgecolor=None, alpha=.01, color="k", ax=ax, legend=False)
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

#### Cell type information 

Identifying the putative ‘cell type’ from the EM morphology is a process that involves both manual and automatic classifications. Subsets of the dataset have been manually classified by anatomists at the Allen Institute, and these ground truth labels used to train and refine different automated ‘feature classifiers’ over time.

The diversity of manual and automated cell type classifications available in the dataset reflect the fact that definitions of ‘cell types’ in the dataset is an active area of research and must be contextualized against the purpose and resolution of the cell-typing being performed.

__Manual Cell Types (V1 Column)__

A subset of nucleus detections in a 100 um column (n=2204) in VISp were manually classified by anatomists at the Allen Institute into categories of cell subclasses, first distinguishing cells into classes of non-neuronal, excitatory and inhibitory. Excitatory cells were separated into laminar sub-classes (L23, L4), 3 sub-types of layer 5 cells (ET, IT, NP) and 2 classes of layer 6 cells (IT, CT). Inhibitory cells were classified into Bipolar (BPC), Basket (BC), Martinotti (MC), or Unsure (Unsure). Those neuronal calls are available from the CAVEclient under the table name `allen_v1_column_types_slanted_ref` which references the nucleus id of the cell.


Non-neuronal manual cells type calls enumerate astrocytes, microglia, pericytes, oligodendrocytes (oligo), and oligodendrocyte precursor cells (OPC), and area available in the table `aibs_column_nonneuronal_ref`.

In [ ]:
manual_ct_df = client.materialize.query_table("allen_v1_column_types_slanted_ref", 
                                              desired_resolution=[1000, 1000, 1000], 
                                              split_positions=True,
                                              merge_reference=False)

print(len(manual_ct_df))
manual_ct_df.head(5)

__Automated Cell Type classification (Soma-nucleus model)__

Models were trained based upon the manual Column Neuron labels, as described in [Elabbady et al. Nature 2025](https://www.nature.com/articles/s41586-024-07765-7). Each nucleus was analyzed for a variety of features, and a model trained on and independent dataset to distinguish neurons from non-neuronal detections. Non-neuron detections include both glial cells and false positive detections. The nucleus segmentation detected 171,818 connected components of nucleus objects, this model detected 82K neurons. Evaluation of this model on 1,316 cells in the volume shows the model has a recall of 99.6% for neurons, and a precision of 96.9%. All nucleus detections and the results of this model can be queried and linked to the cellular segmentation using the CAVEclient with the table name nucleus_neuron_svm.

Combining those features with the nucleus features we trained a hierachical model on the manual labels to predict cell-classes and sub-classes across a large number of neurons. This is available as CAVE table `aibs_metamodel_celltypes_v661`.

In [ ]:
ct_df = client.materialize.query_table("aibs_metamodel_celltypes_v661", 
                                       desired_resolution=[1000, 1000, 1000], 
                                       split_positions=True,
                                       merge_reference=False,
                                      )
print(len(ct_df))
ct_df.head(5)

Reference annotations contain `target_id` to merge them onto the table they target (here: the nucleus table). But do not worry, CAVE automatically merges them onto their target table by default (`merge_reference=True`):

In [ ]:
ct_df = client.materialize.query_table("aibs_metamodel_celltypes_v661", 
                                       desired_resolution=[1000, 1000, 1000], 
                                       split_positions=True,
                                       merge_reference=True,
                                       # select_columns=['target_id','pt_root_id','volume','pt_position','cell_type','classification_system']
                                      )

# remove segments with multiple cell bodies
ct_df = ct_df.drop_duplicates("pt_root_id", keep=False)
ct_df.head(5)

The reference table added two additional data columns: `classification_system` and `cell_type`. The `classification_system` divides the cells into excitatitory and inhibitory neurons as well as non-neuronal cells. `cell_type` provides lower level cell annotations.

In [ ]:
ct_df["classification_system"].value_counts()

In [ ]:
ct_df["cell_type"].value_counts()

#### Location vs depth + Cell type


Because the cell type table contains the information about the nuclei, we can use it to plot the locations of all cell bodies as well and label them by type.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=ct_df, x="volume", y="pt_position_y", s=20, edgecolor=None, alpha=.1, color="k", ax=ax, 
                legend=True, hue="classification_system")
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

Soma volume is one of the distinguishing features used to classify cells into their morphological subtypes, and as expected this shows up in the plot of soma size by depth.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6), dpi=150)
ax.tick_params(labelsize=14)
sns.scatterplot(data=ct_df, x="volume", y="pt_position_y", s=20, edgecolor=None, alpha=.1, color="k", ax=ax, 
                hue_order = ct_df.sort_values('cell_type').cell_type.unique(),
                legend=True, hue="cell_type")
ax.invert_yaxis()
ax.set_xlabel("Volume ($\mu m^3$)", fontsize=16)
ax.set_ylabel("Depth ($\mu m$)", fontsize=16)
ax.set_xlim(0, 500)
plt.show()

### Query 2: synapses and connectivity

While synapses are stored as any other table in the database, in this case `synapses_pni_2`, this table is much larger than any other table at more than 337 million rows:

In [ ]:
client.materialize.get_annotation_count('synapses_pni_2')

While we can query the synapse table directly, this is generally not recommended. It is too large to query all at once. CAVE limits to queries to 500,000 rows at once and will display a warning when that happens. Here, we demonstrate this with the limit set to 10:

In [ ]:
synapse_table_name = client.info.get_datastack_info()["synapse_table"]
syn_df = client.materialize.query_table(synapse_table_name, limit=10, desired_resolution=[1, 1, 1], split_positions=True)
syn_df

Instead, you have several options for querying cells of interest:

* Specifying the `pre_ids`
* Specifying the `post_ids`
* Searching within a `bounding_box`
* Using a combination of the above to iterate through large numbers of neurons


#### Query synapses given `pt_root_id` of interest

The `synapse_query` function allows you to query the synapse table in a more convenient way than most other tables.
In particular, the `pre_ids` and `post_ids` let you specify which root id (or collection of root ids) you want to query, with pre_ids indicating the collection of **presynaptic neurons** and post_ids the collection of **postsynaptic neurons**.

Using both `pre_ids` and `post_ids` in one call is effectively a logical AND, returning only those synapses from neurons in the list of `pre_ids` that target neurons in the list of `post_ids`.

Let's look at our example cell from before:

In [ ]:
# Pick example cell
root_id = 864691135572530981

# Query synapse table with synapse_query()
input_syn_df = client.materialize.synapse_query(post_ids=root_id)

print(f"Total number of input synapses for {root_id}: {len(input_syn_df)}")
input_syn_df.head()

Getting the outputs of the cell is similar, except we use the argument `pre_ids`

In [ ]:
# Query synapse table with synapse_query()
output_syn_df = client.materialize.synapse_query(pre_ids=root_id)

print(f"Total number of output synapses for {root_id}: {len(output_syn_df)}")
output_syn_df.head()

#### Overlay synapses on skeleton

Because these measures come from the same physical space, we can overlay the **skeleton** and the **synapses** of a neuron. Knowing where the synapses are along the **branches of a dendrite or axon** is a common features used for analysis.

More about operating on the arbor of a neuron skeleton is available from the [Skeleton and Mesh Tutorials](https://tutorial.microns-explorer.org/quickstart_notebooks/07-cave-download-skeleton.html)

In [ ]:
# select the example cell from neuroglancer
root_id = 864691135572530981

# Query the skeleton from CAVEclient
sk_d = client.skeleton.get_skeleton(root_id, output_format="dict")

# Query synapse table with synapse_query()
output_syn_df = client.materialize.synapse_query(pre_ids=root_id,
                                                split_positions=True,
                                                desired_resolution=[1,1,1] # same resolution as the skeleton
                                                )

In [ ]:
# Plot your skeleton and overlay synapses
ax = plot_skeleton(sk_d)

sns.scatterplot(
    data=output_syn_df,
    x="ctr_pt_position_x",
    y="ctr_pt_position_y",
    s=5,
    color="b",
    ax=ax,
    edgecolor=None,
    zorder=100,
)

plt.show()

Note that synapse queries always return the list of every synapse between the neurons in the query, even if there are multiple synapses between the same pair of neurons.

A common pattern to generate a list of connections between unique pairs of neurons is to group by the root ids of the presynaptic and postsynaptic neurons and then count the number of synapses between them.
For example, use `pandas.groupby()` to get the number of synapses from this neuron onto every other neuron:

In [ ]:
# get count of synapses between presynaptic and postsynaptic partners
output_connectivity = (output_syn_df.groupby(['pre_pt_root_id', 'post_pt_root_id'])
 .count()[['id']]
 .rename(columns={'id': 'syn_count'})
 .sort_values( by='syn_count', ascending=False,)
)

output_connectivity
# Note that the 'id' part here is just a way to quickly extract one column. This could be any of the remaining column names, 
# but `id` is often convenient because it is common to all tables.

#### Plot output target distribution

Taking this connectivity information from one neuron, **quantify the synapses it makes onto all cell types**. This is the same process that happens to make the Dash App plots.

Note: here we work with only *unique* connections, even where there is more than one synapse between them

In [ ]:
# First, find the unique postsynaptic partners
unique_postsyn_partners = output_syn_df.post_pt_root_id.unique()

# Select the cell types of the postsynaptic partners
ct_postsyn_partners = ct_df.loc[ct_df.pt_root_id.isin(unique_postsyn_partners)]

# Make a bar plot of the connections to the postsynaptic cell types
ct_summary = (pd.DataFrame(ct_postsyn_partners.value_counts('cell_type'))
              .sort_values('cell_type')
              .reset_index()
             )

f, ax = plt.subplots(1,1, figsize=(6, 6),dpi=150)
sns.barplot(ct_summary, x='cell_type', y='count', hue='cell_type', palette='tab20', ax=ax)
    
# Add labels to the bars
for i, r in ct_summary.iterrows():
    ax.text(i, r['count']-.1, round(r['count'], 2), 
            color='white', ha='center', va='top')

plt.ylabel('Count of unique connections', fontsize = 12, labelpad=12) 
plt.xlabel('Cell Type, Postsynaptic (all cells)', fontsize = 12, labelpad=12) 
plt.xticks(rotation=-30, ha='left')

sns.despine()

plt.show()

### Query 3: Proofread connectivity

Proofreading is necessary to obtain accurate reconstructions of a cell. In the MICrONS dataset, the general rule is that dendrites onto cells with a _single_ cell body are sufficiently proofread to trust synaptic connections onto a cell. Axons on the other hand require so much proofreading that only ~1800 cells have axons such that their outputs should be used for analysis.

For more about proofreading, versioning, and other important information, see: [Proofreading and Data Quality](https://tutorial.microns-explorer.org/proofreading.html). For today we will treat all axons that are at least 'axon_partially_extended` as equally trustworthy. This may not be the best choice depdending on the specfici analysis. Reach out to Bethanny with questions for more.

This information is available in the CAVE table: `proofreading_status_and_strategy`


In [ ]:
proof_all_df = client.materialize.query_table("proofreading_status_and_strategy")

In [ ]:
proof_all_df["strategy_axon"].value_counts()

### Filtering Queries by proofreading status
We can filter our query to only return rows that match a condition by adding a filter to our query:

In [ ]:
proof_axon_df = client.materialize.query_table("proofreading_status_and_strategy", 
                                               filter_in_dict={"strategy_axon": ["axon_partially_extended", "axon_fully_extended", "axon_interareal"]}, 
                                               desired_resolution=[1, 1, 1], 
                                               split_positions=True)
proof_axon_df.tail()

A more unified filter interface is available through a "table manager" interface.

Rather than passing a table name to the `query_table` function, `client.materialize.tables` has a subproperty for each table in the database that can be used to filter that table.

The general pattern for usage is

``` python
client.materialize.tables.{table_name}({filter options}).query({format and timestamp options})
```

where `{table_name}` is the name of the table you want to filter, `{filter options}` is a collection of arguments for filtering the query, and `{format and timestamp options}` are those parameters controlling the format and timestamp of the query.

With this, we can easily query all proofread cells with proofread axons:

In [ ]:
proof_axon_df = client.materialize.tables.proofreading_status_and_strategy(strategy_axon=["axon_partially_extended", "axon_fully_extended", "axon_interareal"]).query(
    select_columns=['pt_root_id','status_axon','status_dendrite','strategy_axon','strategy_dendrite'],
)
proof_axon_df.tail()

#### Collect synapses between proofread cells. 

We can now collect the synapses between a proofread cells to form a 'connectivity matrix', or the known connections from one cell to every other cell. 

In its square form, a **closed set**, this is connectivity forms an Adjacency Matrix, or a graph representation of the network connectivity. Here we look at the connectivity between proofread excitatory neurons.

In [ ]:
# Collect the list of root ids that are proofread and have a cell type
exc_ct_df = ct_df.query("classification_system=='excitatory_neuron'")

proofread_exc_ids = exc_ct_df.loc[exc_ct_df.pt_root_id.isin(proof_axon_df.pt_root_id.values), 'pt_root_id'].values
len(proofread_exc_ids)

In [ ]:
%%time

# Collect all synapses between proofread cells. This may take several minutes to run
proof_proof_syn_df = client.materialize.synapse_query(
    pre_ids=proofread_exc_ids,
    post_ids=proofread_exc_ids,
)
print(len(proof_proof_syn_df))

This gives us the 'long' version of the synapse table, where every row is a synapse. Now we will `pivot` this table into a square form, where both rows and columns are unique cells that form connections. 

In [ ]:
# pivot the synapse table
syn_mat = proof_proof_syn_df.pivot_table(index="pre_pt_root_id", 
                                         columns="post_pt_root_id",
                                         values="size", 
                                         aggfunc=lambda x: float(np.sum(x) > 0)).fillna(0)
syn_mat = syn_mat.reindex(columns=np.array(syn_mat.index))

This gives us a sparse matrix that is mostly zeros, which we can visualize as a connectivity heatmap

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat, cmap="gray_r", 
            xticklabels=[], yticklabels=[], cbar=None,
            ax=ax, 
            square=True,
           )
plt.show()


#### Sort synapse matrix with cell types
Let’s combine the synaptic connecitivity with the cell type information. Below we provide logic for sorting a connectivity matrix using a list of labels.

In [ ]:
def sort_matrix_by_types(mat: pd.DataFrame, 
                         labels: pd.DataFrame, 
                         label_type_col: str = "cell_type", 
                         label_id_col: str = "pt_root_id", 
                         post_labels: pd.DataFrame = None, 
                         post_label_type_col: str = None, 
                         post_label_id_col: str = None):
    """Sorts (synapse) matrix by labels.

    This function assumes a square synapse matrix!

    Args:
        mat: synapse matrix as pandas DataFrame
        labels: DataFrame with labels, e.g. the output of client.materialize.query_table('aibs_metamodel_celltypes_v661')
        label_type_col: column name in labels for cell types
        label_id_col: column name in labels for root ids
        post_labels: DataFrame with labels, e.g. the output of client.materialize.query_table('aibs_metamodel_celltypes_v661')
        post_label_type_col: column name in labels for cell types
        post_label_id_col: column name in labels for root ids

    Returns:
        mat_sorted: sorted matrix
        mat_labels: sorted labels; has the same length as matrix
    """
    
    if post_labels is None:
        post_labels = labels
    if post_label_type_col is None:
        post_label_type_col = label_type_col
    if post_label_id_col is None:
        post_label_id_col = label_id_col
        
    mat_sorted = mat.copy()
    
    pre_mat_labels = np.array(labels.set_index(label_id_col).loc[mat_sorted.index][label_type_col])
    pre_sorting = np.argsort(pre_mat_labels)

    post_mat_labels = np.array(post_labels.set_index(post_label_id_col).loc[mat_sorted.T.index][post_label_type_col])
    post_sorting = np.argsort(post_mat_labels)

    mat_sorted = mat_sorted.iloc[pre_sorting].T.iloc[post_sorting].T

    return mat_sorted, pre_mat_labels[pre_sorting], post_mat_labels[post_sorting]

In [ ]:
# sort the matrix by cell types to render sensibly in heatmap
syn_mat_ct, syn_mat_cell_types, _ = sort_matrix_by_types(syn_mat, exc_ct_df, label_type_col="cell_type")

In [ ]:
import matplotlib

# add colormap for cell type
cts, ct_idx = np.unique(syn_mat_cell_types, return_inverse=True)
ct_colors = plt.get_cmap("tab10")(ct_idx)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
sns.heatmap(syn_mat_ct, cmap="gray_r", xticklabels=[], yticklabels=[], 
            ax=ax, square=True,
            cbar=None)


# Adding row and column colors for cell types
for i, color in enumerate(ct_colors):
    ax.add_patch(plt.Rectangle(xy=(-0.01, i), width=0.01, height=1, color=color, lw=0,
                               transform=ax.get_yaxis_transform(), clip_on=False))

for i, color in enumerate(ct_colors):
    ax.add_patch(plt.Rectangle(xy=(i, 1), height=0.01, width=1, color=color, lw=0,
                               transform=ax.get_xaxis_transform(), clip_on=False))

# add a legend for the cell types
legend_elements = [matplotlib.lines.Line2D([0], [0], color=plt.get_cmap("tab10")(i), label=ct) for i, ct in enumerate(cts)]
plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1.3, 1), title="cell types")
plt.show()

# Advanced Neuroglancer

You can actually overlay many of these properties in neuroglancer, including:

* Skeletons: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6025750544121856 
* Nucleus segmentation: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/4846103861133312 
* Cell Types: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5505896963637248
* Proofreading Status: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/5626062229733376
* Synapses: https://spelunker.cave-explorer.org/#!middleauth+https://global.daf-apis.com/nglstate/api/v1/6395755509055488

And any combination thereof. Tip: drag layers between diferent neuroglancer windows to view overlaid

